In [24]:
import openslide
from openslide import lowlevel as openslide_ll
from ctypes import c_uint32, POINTER, cast, addressof, c_void_p, byref, c_uint8, sizeof, c_uint16
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Optional
from loguru import logger
import re
import cv2
from multiprocessing import RawArray
import concurrent
from concurrent.futures import ProcessPoolExecutor
from concurrent import futures
from tqdm import tqdm
from pathlib import Path
import dask.array as da
import dask
from dask.distributed import progress, wait
import torch
from dask.distributed import Client, LocalCluster
import hydra
from hydra.utils import instantiate
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf

from shiprec.run import *
from time import time

# Manually set up hydra
GlobalHydra.instance().clear()
cfg = instantiate(OmegaConf.load("config.yaml"))

slide_file = "/mnt/bulk/gwoelflein/georg-tcga-slides/tcga_brca_test/TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4BD-C6B76294C291.svs"
check_status = False

In [9]:
client, pipeline_setup = setup(cfg)
client

/mnt/bulk/gwoelflein/georg-transformers/shiprec/env/lib64/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32999 instead
  warnings.warn(


2023-08-14 11:37:36.427 | INFO     | Started Dask client with 17 workers, 1 of which are GPU workers
2023-08-14 11:37:36.428 | INFO     | Access the Dask dashboard at http://127.0.0.1:32999/status
2023-08-14 11:37:36.445 | DEBUG    | Fitting Macenko normalizer to template
2023-08-14 11:37:36.528 | DEBUG    | Loading feature extraction model


/mnt/bulk/gwoelflein/georg-transformers/shiprec/env/lib64/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2023-08-14 11:37:41.331 | DEBUG    | Loaded and scattered feature extractor to workers
2023-08-14 11:37:41.332 | INFO     | Setup completed, took 6.96 seconds


<Client: 'tcp://127.0.0.1:41451' processes=17 threads=17, memory=506.64 GiB>

In [25]:
"""Process a slide and save the features to disk."""
start_time = time()
slide_file = Path(slide_file)
if client is None:
    client = get_client()
    wait = secede_wait_rejoin  # TODO: check if this is faster

optimize_graph = cfg.cluster.optimize_graph

normalizer, model, has_gpu_devices = pipeline_setup

futures_to_cancel = []

# Check if the slide has already been processed
status_file = Path(cfg.output.path) / "status" / f"{slide_file.stem}.done"
if check_status and status_file.exists():
    logger.info(f"Skipping {slide_file.stem} because it has already been processed")
    raise

def write_status_file(message: str = "done"):
    status_file.parent.mkdir(parents=True, exist_ok=True)
    with status_file.open("w") as f:
        f.write(message)

# Load the slide
try:
    slide = load_slide(slide_file, target_mpp=cfg.pipeline.target_mpp, backend=cfg.input.slide_backend)
except MPPExtractionError:
    logger.warning(f"Could not extract MPP for {slide_file}, skipping")
    write_status_file("mpp_extraction_error")
    raise

# Split the slide into patches
patches, patch_coords = split_slide_into_patches(slide, patch_size=cfg.pipeline.patch_size)
patch_grid_shape = patches.shape[:2]
patches, patch_coords = flatten_patches_and_coords(patches, patch_coords)
patches = patches.rechunk(("auto", -1, -1, -1))  # TODO: remove?
logger.debug(f"Loaded patches will be of shape {patches.shape} with chunks {patches.chunks}")

# Filter out background patches
patch_mask = get_canny_foreground_mask(patches)
patches = patches[patch_mask]
patch_coords = patch_coords[patch_mask]

# Cache the patches and patch coordinates
patches, patch_coords, patch_mask = client.persist(
    (patches, patch_coords, patch_mask), optimize_graph=optimize_graph
)
futures_to_cancel.extend((patches, patch_coords, patch_mask))
logger.debug("Loading patches, filtering background")
wait((patches, patch_coords))
# tqdm_dask((patches, patch_coords), desc="Loading patches, filtering background")

# We need deterministic chunk sizes for the following steps
patches.compute_chunk_sizes()
patch_coords.compute_chunk_sizes()
patch_mask.compute_chunk_sizes()
# patches = patches.rechunk(
#     (cfg.pipeline.stain_normalization.batch_size, cfg.pipeline.patch_size, cfg.pipeline.patch_size, 3)
# )
logger.debug(
    f"Reduced number of patches from {np.prod(patch_grid_shape)} to {patches.shape[0]}; chunked as {patches.chunks[0]}"
)

# Compute the patch index grid locally
patch_grid = np.zeros(patch_grid_shape, dtype=int) - 1
patch_grid = np.reshape(patch_grid, (-1,))
computed_patch_mask = patch_mask.compute()  # should be cheap
patch_grid[computed_patch_mask] = np.arange(computed_patch_mask.sum())
patch_grid = np.reshape(patch_grid, patch_grid_shape)

# Perform Macenko normalization
if cfg.pipeline.stain_normalization.enabled:
    normalized = normalizer.normalize_flattened_image(patches.reshape(-1, 3))
    normalized = normalized.reshape(patches.shape)
    # normalized = normalized.rechunk(
    #     (cfg.pipeline.feature_extraction.batch_size, cfg.pipeline.patch_size, cfg.pipeline.patch_size, 3)
    # )
    normalized = normalized.persist(optimize_graph=optimize_graph)
    futures_to_cancel.append(normalized)
    logger.debug("Performing Macenko normalization")
    wait(normalized)
    # tqdm_dask(normalized, desc="Macenko normalization")
else:
    normalized = patches

# Extract features
features = extract_features(model, normalized, use_cuda=has_gpu_devices)

# Cache the features
# NOTE: if optimize_graph=True, then the graph optimization might move the model to a non-GPU worker (but
# if we don't have any GPU workers, then this is not a problem)
features = features.persist(optimize_graph=not has_gpu_devices)
futures_to_cancel.append(features)
logger.debug("Extracting features")
wait(features)

2023-08-14 11:41:35.423 | DEBUG    | MPP successfully extracted using extract_mpp_from_properties: 0.250
2023-08-14 11:41:35.424 | DEBUG    | Slide has 4 levels with following downsamples: {0: 1.0, 1: 4.0, 2: 16.0, 3: 32.0}
2023-08-14 11:41:35.424 | DEBUG    | Using level 1 with level_mpp=1.000 for slide_mpp=0.250 and target_mpp=1.143
2023-08-14 11:41:35.445 | DEBUG    | Loaded patches will be of shape (8960, 224, 224, 3) with chunks ((672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 224), (224,), (224,), (3,))
2023-08-14 11:41:35.483 | DEBUG    | Loading patches, filtering background
 cuInit Failed, error CUDA_ERROR_NO_DEVICE
 cuFile initialization failed
 cuInit Failed, error CUDA_ERROR_NO_DEVICE
 cuFile initialization failed
 cuInit Failed, error CUDA_ERROR_NO_DEVICE
 cuFile initialization failed
 cuInit Failed, error CUDA_ERROR_NO_DEVICE
 cuFile initialization failed
 cuInit Failed, error CUDA_ERROR_NO_DEVICE
 cuFile initialization failed
 cuInit Failed, error CUDA_

KeyboardInterrupt: 